In [1]:
import pandas as pd
from  more_itertools import unique_everseen
import codecs
import re
from tqdm import tqdm, tqdm_notebook
tqdm_notebook().pandas()

<ipython-input-1-25c0eb0a05c5>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tqdm_notebook().pandas()


C:\Users\Julia\anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
def list_to_file(filepath, list_, codec):
    with codecs.open(filepath, 'w', codec) as file:
        for line in list_:
            line = line + '\n'
            file.write(line)
            
def read_list(filepath, codec):
    with codecs.open(filepath, 'r', codec) as file:
        data = file.readlines()
        data = [line.rstrip() for line in data]
    return data

def recursive_flatten_generator(array):
    lst = []
    for i in array:
        if isinstance(i, list):
            lst.extend(recursive_flatten_generator(i))
        else:
            lst.append(i)
    return lst

In [4]:
data = pd.read_csv('patients_with operations_dataset_clean.txt', encoding='cp1251', sep='\t', engine='python', header=None, error_bad_lines=False,parse_dates=[3])
data.columns = ['index', 'patient', 'episode','date', 'time','clinitian', 'depart', 'hosp','specialty','appoint','document','text','vyp']
intra_docs = read_list('intra_docs.txt','utf-8')
hgb_keys = ['hgb','hb','гемоглобин']

In [5]:
data['text'] = data.apply(lambda x: re.sub(r'293pan\d+', '', x['text']), axis=1)
data['text'] = data.apply(lambda x: re.sub(r'10sup\d+', '', x['text']), axis=1)
data['text'] = data.apply(lambda x: re.sub(r'\s+', ' ', x['text']), axis=1)

In [6]:
data.head()

,index,patient,episode,date,time,clinitian,depart,hosp,specialty,appoint,document,text,vyp
0,0,GACAAGc,GACAAGcAAAAIe10:30-uAA,2009-03-18,10:30b 11:00,Наумова_Людмила_Васильевна,Кар,Центр_Алмазова,Врач-кардиолог,"Прием_(осмотр,_консультация)_врача-кардиолога_...",ДИАГНОЗ_АМБУЛАТОРНЫЙ,ДИАГНОЗ_ПРЕДВАРИТЕЛЬНЫЙ_ОСНОВНОЙ I20.8 Другие ...,выполнено
1,1,GACAAGc,GACAAGcAAAAIxN20288uAA,2009-04-06,10:55,Наумова_Людмила_Васильевна,Кар,Центр_Алмазова,Врач-кардиолог,"Прием_(осмотр,_консультация)_врача-кардиолога_...",ДИАГНОЗ_АМБУЛАТОРНЫЙ,ДИАГНОЗ_ПРЕДВАРИТЕЛЬНЫЙ_ОСНОВНОЙ I20.8 Другие ...,выполнено
2,2,GACAAGc,GACAAGcAAAAIxN20288{AD,2009-04-06,10:55,Наумова_Людмила_Васильевна,Кар,Центр_Алмазова,Врач-кардиолог,"Прием_(осмотр,_консультация)_врача-кардиолога_...",РЕКОМЕНДАЦИИ,"Липидный спектр, калий, креатинин, МНО, сахар,...",выполнено
3,3,GACAAGc,GACAAGcAAAAJN15:00-iAD,2009-04-22,15:00-15:30,Дзюбенко_Наталья_Александровна,ФД,Центр_Алмазова,Врач_ультразвуковой_диагностики,Эхокардиография,ЭХОКАРДИОГРАФИЧЕСКОЕ_ИССЛЕДОВАНИЕ_РЕЗУЛЬТАТ,"ЧСС 64 уд.в мин., Ритм синусовый",выполнено
4,4,GACAAGc,GACAAGcAAAAJN15:00-iAG,2009-04-22,15:00-15:30,Дзюбенко_Наталья_Александровна,ФД,Центр_Алмазова,Врач_ультразвуковой_диагностики,Эхокардиография,ЭХОКАРДИОГРАФИЧЕСКОЕ_ИССЛЕДОВАНИЕ_РЕЗУЛЬТАТ,"Аорта 41 мм стенки уплотнены ,диаметр расшире...",выполнено


In [7]:
patients = list(unique_everseen(list(data.patient.values)))
len(patients)

455

In [8]:
pat_data = data.loc[data['patient']==patients[0]]
oper = pat_data.loc[pat_data['document'].isin(intra_docs)]
oper.date.value_counts() #операция одна, дата одна

2013-12-17    7
Name: date, dtype: int64

In [77]:
oper_date = oper.date.value_counts().keys().to_list()
oper_time = oper.time.value_counts().keys().to_list()
oper_time

['13:17']

In [52]:
hgb_rgx = r'(?i)(?:hb|hb\s+|hgb|hgb\s+|гемоглобин|гемоглобин\s+|гемоглобин b)([-+]?(?:\d+(?:\.\d*)?|\.\d+)(?:[eE][-+]?\d+)?)'
def hgb_checker(string, hgb_rgx):
    return re.findall(hgb_rgx, string)

In [53]:
new_data=data
new_data['hgb'] = new_data.progress_apply(lambda x: hgb_checker(x['text'], hgb_rgx), axis=1)
#new_data['hgb'] = new_data.apply(lambda x: str(x['hgb']),axis=1)

In [55]:
new_data['hgb'] = new_data.progress_apply(lambda x: list(unique_everseen(x['hgb'])), axis=1)

In [60]:
new_data['hgb'] = new_data['hgb'].apply(lambda x: x if (len(x)==1) else 0)
new_data = new_data.loc[new_data['hgb']!=0]
new_data['hgb'] = new_data['hgb'].apply(lambda x: x[0])
new_data.head()

<ipython-input-60-750348b15691>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['hgb'] = new_data['hgb'].apply(lambda x: x[0])


,index,patient,episode,date,time,clinitian,depart,hosp,specialty,appoint,document,text,vyp,hgb
384,435,GACAAGc,GACAAGcAARAAQ03:26AaAj,2013-12-18,03:26,Дюжилов_Алексей_Вячеславович,ОАиР_с_ПИТ_№2_(Аккуратова),Центр_Алмазова,Врач-анестезиолог-реаниматолог,Осмотр_(консультация)_врача-анестезиолога-реан...,ДНЕВНИК_НАБЛЮДЕНИЙ_В_РЕАНИМАЦИОННОМ_ОТДЕЛЕНИИ,"Уровень pH a 7,39 pH 7,33 PaO2 119_мм.рт.ст. ...",выполнено,97
389,442,GACAAGc,GACAAGcAARAAQ08:01AaAD,2013-12-18,08:01,Дюжилов_Алексей_Вячеславович,ОАиР_с_ПИТ_№2_(Аккуратова),Центр_Алмазова,Врач-анестезиолог-реаниматолог,Осмотр_(консультация)_врача-анестезиолога-реан...,ДНЕВНИК_НАБЛЮДЕНИЙ_В_РЕАНИМАЦИОННОМ_ОТДЕЛЕНИИ,Осмотр дежурного реаниматолога Состояние тяже...,выполнено,90
402,458,GACAAGc,GACAAGcAARAAQ10:30AcAm,2013-12-18,10:30,Ташханов_Дмитрий_Маратович,ОАиР_с_ПИТ_№2_(Аккуратова),Центр_Алмазова,Врач-анестезиолог-реаниматолог,Суточное_наблюдение_врача-реаниматолога,ПЕРЕВОДНОЙ_ЭПИКРИЗ_ИЗ_РЕАНИМАЦИОННОГО_ОТДЕЛЕНИЯ,"Уровень pH a 7,37 pH 7,32 PaO2 97 PvO2 38 PaC...",выполнено,89
445,523,GACAAGc,GACAAGcAARAAS14:33AoAa,2013-12-20,14:33,Олещук_Анна_Никитична,ССХ_№3_(Пархоменко),Центр_Алмазова,Врач-интерн,Курация_пациента_лечащим_врачом,ДНЕВНИК_ССХ3,Динамика состояния Состояние пациента без отр...,выполнено,85
465,545,GACAAGc,GACAAGcAARAAV10:57AoAa,2013-12-23,10:57,Зубова_Елена_Сергеевна,ССХ_№3_(Пархоменко),Центр_Алмазова,Врач-сердечно-сосудистый_хирург,Курация_пациента_лечащим_врачом,ДНЕВНИК_ССХ3,Динамика состояния Отмечается положительная д...,выполнено,89.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
591899,1765613,GACAdжХ,GACAdжХAABAAG16:54A!AY,2018-11-07,16:54,Ташханов_Дмитрий_Маратович,ОАиР_с_ПИТ_№2_(Аккуратова),Центр_Алмазова,Врач-анестезиолог-реаниматолог,Требование_на_выдачу_ГТС,ТРЕБОВАНИЕ_на_выдачу_ГТС,Трансфузионный анамнез Без_особенностей Ht 0....,выполнено,82.0
591986,1765705,GACAdжХ,GACAdжХAABAAH07:32A!AY,2018-11-08,07:32,Ташханов_Дмитрий_Маратович,ОАиР_с_ПИТ_№2_(Аккуратова),Центр_Алмазова,Врач-анестезиолог-реаниматолог,Требование_на_выдачу_ГТС,ТРЕБОВАНИЕ_на_выдачу_ГТС,Трансфузионный анамнез Без_особенностей Ht 0....,выполнено,81.0
592020,1765749,GACAdжХ,GACAdжХAABAAH10:54A#Ap,2018-11-08,10:54,Нефедова_Ирина_Николаевна,ОАиР_с_ПИТ_№2_(Аккуратова),Центр_Алмазова,Врач-анестезиолог-реаниматолог,Суточное_наблюдение_врача-реаниматолога,ПЕРЕВОДНОЙ_ЭПИКРИЗ_ИЗ_РЕАНИМАЦИОННОГО_ОТДЕЛЕНИЯ,"Уровень / pHa 7.42, pHv 7.35, pO2a 164.0, pO2...",выполнено,95.0
592647,1767837,GACAdтА,GACAdтАAABAAC10:06A!AY,2018-10-30,10:06,Титкова_Елена_Анатольевна,ОТТ,Центр_Алмазова,Заведующий,Требование_на_выдачу_ГТС,ТРЕБОВАНИЕ_на_выдачу_ГТС,Трансфузионный анамнез Без_особенностей Ht 0....,выполнено,115.0


In [28]:
#new_data['hgb'] = new_data.progress_apply(lambda x: ' '.join(x['hgb']), axis=1)

In [62]:
hgb_data = new_data.loc[new_data['hgb']!='']
hgb_data['hgb'] = hgb_data.apply(lambda x:re.sub(r'-', '', x['hgb']), axis=1)
hgb_data

,index,patient,episode,date,time,clinitian,depart,hosp,specialty,appoint,document,text,vyp,hgb
384,435,GACAAGc,GACAAGcAARAAQ03:26AaAj,2013-12-18,03:26,Дюжилов_Алексей_Вячеславович,ОАиР_с_ПИТ_№2_(Аккуратова),Центр_Алмазова,Врач-анестезиолог-реаниматолог,Осмотр_(консультация)_врача-анестезиолога-реан...,ДНЕВНИК_НАБЛЮДЕНИЙ_В_РЕАНИМАЦИОННОМ_ОТДЕЛЕНИИ,"Уровень pH a 7,39 pH 7,33 PaO2 119_мм.рт.ст. ...",выполнено,97
389,442,GACAAGc,GACAAGcAARAAQ08:01AaAD,2013-12-18,08:01,Дюжилов_Алексей_Вячеславович,ОАиР_с_ПИТ_№2_(Аккуратова),Центр_Алмазова,Врач-анестезиолог-реаниматолог,Осмотр_(консультация)_врача-анестезиолога-реан...,ДНЕВНИК_НАБЛЮДЕНИЙ_В_РЕАНИМАЦИОННОМ_ОТДЕЛЕНИИ,Осмотр дежурного реаниматолога Состояние тяже...,выполнено,90
402,458,GACAAGc,GACAAGcAARAAQ10:30AcAm,2013-12-18,10:30,Ташханов_Дмитрий_Маратович,ОАиР_с_ПИТ_№2_(Аккуратова),Центр_Алмазова,Врач-анестезиолог-реаниматолог,Суточное_наблюдение_врача-реаниматолога,ПЕРЕВОДНОЙ_ЭПИКРИЗ_ИЗ_РЕАНИМАЦИОННОГО_ОТДЕЛЕНИЯ,"Уровень pH a 7,37 pH 7,32 PaO2 97 PvO2 38 PaC...",выполнено,89
445,523,GACAAGc,GACAAGcAARAAS14:33AoAa,2013-12-20,14:33,Олещук_Анна_Никитична,ССХ_№3_(Пархоменко),Центр_Алмазова,Врач-интерн,Курация_пациента_лечащим_врачом,ДНЕВНИК_ССХ3,Динамика состояния Состояние пациента без отр...,выполнено,85
465,545,GACAAGc,GACAAGcAARAAV10:57AoAa,2013-12-23,10:57,Зубова_Елена_Сергеевна,ССХ_№3_(Пархоменко),Центр_Алмазова,Врач-сердечно-сосудистый_хирург,Курация_пациента_лечащим_врачом,ДНЕВНИК_ССХ3,Динамика состояния Отмечается положительная д...,выполнено,89.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
591899,1765613,GACAdжХ,GACAdжХAABAAG16:54A!AY,2018-11-07,16:54,Ташханов_Дмитрий_Маратович,ОАиР_с_ПИТ_№2_(Аккуратова),Центр_Алмазова,Врач-анестезиолог-реаниматолог,Требование_на_выдачу_ГТС,ТРЕБОВАНИЕ_на_выдачу_ГТС,Трансфузионный анамнез Без_особенностей Ht 0....,выполнено,82.0
591986,1765705,GACAdжХ,GACAdжХAABAAH07:32A!AY,2018-11-08,07:32,Ташханов_Дмитрий_Маратович,ОАиР_с_ПИТ_№2_(Аккуратова),Центр_Алмазова,Врач-анестезиолог-реаниматолог,Требование_на_выдачу_ГТС,ТРЕБОВАНИЕ_на_выдачу_ГТС,Трансфузионный анамнез Без_особенностей Ht 0....,выполнено,81.0
592020,1765749,GACAdжХ,GACAdжХAABAAH10:54A#Ap,2018-11-08,10:54,Нефедова_Ирина_Николаевна,ОАиР_с_ПИТ_№2_(Аккуратова),Центр_Алмазова,Врач-анестезиолог-реаниматолог,Суточное_наблюдение_врача-реаниматолога,ПЕРЕВОДНОЙ_ЭПИКРИЗ_ИЗ_РЕАНИМАЦИОННОГО_ОТДЕЛЕНИЯ,"Уровень / pHa 7.42, pHv 7.35, pO2a 164.0, pO2...",выполнено,95.0
592647,1767837,GACAdтА,GACAdтАAABAAC10:06A!AY,2018-10-30,10:06,Титкова_Елена_Анатольевна,ОТТ,Центр_Алмазова,Заведующий,Требование_на_выдачу_ГТС,ТРЕБОВАНИЕ_на_выдачу_ГТС,Трансфузионный анамнез Без_особенностей Ht 0....,выполнено,115.0


In [67]:
#hgb_data.to_csv(r'check_hgb.txt', index=None, sep='\t')